In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [ ]:
train_dir = '/content/drive/MyDrive/Artificial Intelligence part_1/Train'
test_dir = '/content/drive/MyDrive/Artificial Intelligence part_1/Test'
val_dir = '/content/drive/MyDrive/Artificial Intelligence part_1/Validation'


In [ ]:
# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Validation and test data generator (only rescaling)
val_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Flow from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


In [ ]:
model = Sequential([
    # Convolutional Layers
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    # Flatten and Fully Connected Layers
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)


In [ ]:
# Evaluate on test data
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print(f"Test Accuracy: {test_acc:.2f}")


In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
# unseen data


from tensorflow.keras.preprocessing import image

# Load a single image
img_path = '/content/drive/MyDrive/Artificial Intelligence part_1/images.jfif'  # Update path
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predict
prediction = model.predict(img_array)
if prediction[0] > 0.5:
    print("Fake")
else:
    print("Real")


Transfer learning using the Xception model

In [ ]:
import os
import numpy as np
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

###############################         Data processing        #############################

# Data generators with augmentation for training and rescaling for test/validation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load data from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


###################      Build  Xception model   ##################


# Load pre-trained Xception model without the top layers
base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])








#################################     Train the model



history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,  # Adjust based on your needs
    validation_data=val_generator,
    validation_steps=len(val_generator)
)



#################### Evaluate ####################
# Evaluate on test data
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print(f"Test Accuracy: {test_acc:.2f}")





In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training vs Validation Accuracy')
plt.show()


In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training vs Validation Loss')
plt.show()


In [ ]:
from tensorflow.keras.preprocessing import image

# Load and preprocess a single image
img_path = '/content/drive/MyDrive/Artificial Intelligence part_1/Test/Real/real_1001.jpg'  # Update path
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predict
prediction = model.predict(img_array)
if prediction[0] > 0.5:
    print("Fake")
else:
    print("Real")


In [ ]:
for layer in base_model.layers[-20:]:  # Unfreeze the last 20 layers
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Retrain with a lower learning rate
fine_tune_history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=5,  # Few epochs for fine-tuning
    validation_data=val_generator,
    validation_steps=len(val_generator)
)


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Generate predictions
y_true = test_generator.classes
y_pred = model.predict(test_generator)
y_pred_classes = (y_pred > 0.5).astype(int)

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Real', 'Fake'], yticklabels=['Real', 'Fake'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Classification Report
print(classification_report(y_true, y_pred_classes, target_names=['Real', 'Fake']))


#                                       **GAN** MODEL

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Flatten, Dense, Reshape
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns


In [ ]:
# Normalize and augment the dataset
datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)


In [ ]:
def build_generator():
    model = Sequential([
        Dense(8*8*256, activation='relu', input_dim=100),
        Reshape((8, 8, 256)),
        Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', activation='relu'),
        Conv2DTranspose(64, kernel_size=4, strides=2, padding='same', activation='relu'),
        Conv2DTranspose(3, kernel_size=4, strides=2, padding='same', activation='sigmoid')  # Output: 64x64x3
    ])
    return model


In [ ]:
def build_discriminator(input_shape=(64, 64, 3)):
    model = Sequential([
        Conv2D(64, kernel_size=4, strides=2, padding='same', input_shape=input_shape, activation='relu'),
        Conv2D(128, kernel_size=4, strides=2, padding='same', activation='relu'),
        Conv2D(256, kernel_size=4, strides=2, padding='same', activation='relu'),
        Flatten(),
        Dense(1, activation='sigmoid')  # Binary classification: Real or Fake
    ])
    return model


In [ ]:
def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(shape=(100,))
    fake_image = generator(gan_input)
    validity = discriminator(fake_image)
    return Model(gan_input, validity)


In [ ]:
# Initialize models
generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

# Compile models
discriminator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])
gan.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')


In [ ]:
# Training loop
epochs = 700
batch_size = 32
latent_dim = 100

real_label = np.ones((batch_size, 1))  # Labels for real images
fake_label = np.zeros((batch_size, 1))  # Labels for fake images

# Training process
for epoch in range(epochs):
    # Train Discriminator
    real_images = next(train_generator)[0]  # Get a batch of real images
    z = np.random.normal(0, 1, (batch_size, latent_dim))  # Generate random noise
    fake_images = generator.predict(z)  # Generate fake images

    d_loss_real = discriminator.train_on_batch(real_images, real_label)
    d_loss_fake = discriminator.train_on_batch(fake_images, fake_label)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train Generator
    z = np.random.normal(0, 1, (batch_size, latent_dim))
    g_loss = gan.train_on_batch(z, real_label)

    # Print progress
    if epoch % 50 == 0:
        print(f"Epoch {epoch} - D Loss: {d_loss[0]}, G Loss: {g_loss}")



In [ ]:
real_images, real_labels = next(test_generator)
z = np.random.normal(0, 1, (len(real_images), latent_dim))
fake_images = generator.predict(z)

# Evaluate real and fake images using the discriminator
real_preds = discriminator.predict(real_images)
fake_preds = discriminator.predict(fake_images)


In [ ]:
y_true = np.concatenate([np.ones(len(real_preds)), np.zeros(len(fake_preds))])
y_pred = np.concatenate([real_preds, fake_preds])
y_pred_binary = (y_pred > 0.5).astype(int)

cm = confusion_matrix(y_true, y_pred_binary)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Fake', 'Real'], yticklabels=['Fake', 'Real'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
print(classification_report(y_true, y_pred_binary, target_names=['Fake', 'Real']))


In [ ]:
# Plot real images
plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(real_images[i])
    plt.title("Real")
    plt.axis('off')

plt.show()

# Plot fake images
plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(fake_images[i])
    plt.title("Fake")
    plt.axis('off')

plt.show()


In [ ]:
plt.plot(d_loss, label='Discriminator Loss')
plt.plot(g_loss, label='Generator Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training Loss Curves')
plt.show()
